<a href="https://colab.research.google.com/github/jonad/Transformer_net_time_series/blob/master/transformer_dj.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Transformer network for financial trading

![](https://drive.google.com/uc?id=1kCuwH7rOpdqjF9ZPACwIOaW5TpxFQ0cy)

In [0]:
import math, random
import gym
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.autograd as autograd
import torch.nn.functional as F
from torch.distributions import Categorical
import pandas as pd
from scipy import signal 

%matplotlib inline
import matplotlib.pyplot as plt

import os

In [0]:
import torch
from torch.utils.data import Dataset
from sklearn.preprocessing import MinMaxScaler
from math import floor
from torch.utils.data.dataset import random_split
from torch.utils.data import Dataset, DataLoader

class StockDataSet(Dataset):

 def __init__(self, lookback=60, q=19, start_date="2012-01-01", end_date="2019-08-31"):
  data = yf.download("^GSPC", start=start_date, end=end_date, group_by="ticker")
  data['Close+1'] = data.Close.shift(1)
  data['Return'] = (data['Close+1'] - data.Close) / data['Close+1']
  data['Trend'] = (data.Close.shift(-q) - data.Close) / data.Close
  data['Target'] = np.where(data['Trend'] > 0, 1, 0)
  data = data.dropna()
  trend = data["Trend"].values
  trend = np.reshape(trend, (len(trend), 1))
  sc = MinMaxScaler()
  self.inputs = sc.fit_transform(trend)
  target = data["Target"].values
  self.labels = np.reshape(target, (len(target), 1))
  self.lookback = lookback
  #assert len(self.inputs) == len(self.labels)

 def __getitem__(self, index):
  x = self.inputs[index: (index + self.lookback)]
  y = self.labels[index + self.lookback]
  return torch.from_numpy(x).float(), torch.from_numpy(y).float()

 def __len__(self):
  return len(self.inputs) - self.lookback

In [0]:
!pip install yfinance
import yfinance as yf

  Created wheel for yfinance: filename=yfinance-0.1.45-cp36-none-any.whl size=14652 sha256=fd8db50b96671e3465d438762a9be21d01bc858a9fe58b5023560d747dc75bb0
  Stored in directory: /root/.cache/pip/wheels/0c/d1/df/aa9a7744a4ac353cc9a1f2c3aaea7c1f457fc49de4286f2d88
Successfully built yfinance


In [0]:
dataset = StockDataSet()
training_size = floor(0.9 * len(dataset))
validation_size = len(dataset) - training_size
train_dataset, val_dataset = random_split(dataset, [training_size, validation_size])

train_loader = DataLoader(dataset=train_dataset, batch_size=2, shuffle=True)
val_loader = DataLoader(dataset=val_dataset, batch_size=2, shuffle=False)

[*********************100%***********************]  1 of 1 downloaded


In [0]:
for i, (images, labels) in enumerate(train_loader):
    print(images.shape)
    break

torch.Size([2, 60, 1])


In [0]:
class Transformer(nn.Module):
  def __init__(self, sequence_length, output_size, batch_size, d_model=1, nhead=1, num_encoders_layers=6):
    super(Transformer, self).__init__()
    
    self.output_size = output_size
    self.batch_size = batch_size
    self.sequence_length = sequence_length
    self.encoder_layer = nn.TransformerEncoderLayer(d_model, nhead, dim_feedforward=1)
    self.transformer_encoder = nn.TransformerEncoder(self.encoder_layer, num_encoders_layers)
    self.fc = nn.Linear(self.sequence_length, output_size)
    self.sigmoid = nn.Sigmoid()
    
  def forward(self, src):
    out = self.transformer_encoder(src)
    out= out.view(self.batch_size, -1)
    out = self.fc(out)
    sig_out = self.sigmoid(out)
    return sig_out
    
    
    
    

In [0]:
sequenze_length = 60
output_size = 1
batch_size = 3
d_model = 1
nhead = 1
num_encoders_layers = 10

dataset = StockDataSet()
training_size = floor(0.9 * len(dataset))
validation_size = len(dataset) - training_size
train_dataset, val_dataset = random_split(dataset, [training_size, validation_size])

train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(dataset=val_dataset, batch_size=batch_size, shuffle=False)

net = Transformer(sequenze_length, output_size, batch_size, d_model, nhead, num_encoders_layers)

[*********************100%***********************]  1 of 1 downloaded


In [0]:
### training
lr = 0.01
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(net.parameters(), lr=lr)
epochs = 3
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
net.to(device)
counter = 0
for e in range(epochs):
  net.train()
  
  for inputs, labels in train_loader:
    if len(inputs) != batch_size:
          continue
    counter += 1
    net.zero_grad()
    
    # get the output
    inputs = inputs.to(device)
    labels = labels.to(device)
        
    output = net(inputs)
    
    loss = criterion(output.squeeze(), labels.float())
    loss.backward()
    
    optimizer.step()
    
  #get the validation loss
  val_losses = []
  val_acc = []
  net.eval()
  for inputs, labels in val_loader:
    if len(inputs) != batch_size:
          continue
    inputs, labels = inputs.cuda(), labels.cuda()
    output = net(inputs)
    val_loss = criterion(output.squeeze(), labels.float())
    
    output = (output > 0.5).float()
    correct = (output.cpu().numpy().flatten() == \
                   labels.cpu().numpy().flatten()).sum() / output.shape[0]
    val_losses.append(val_loss.item())
    val_acc.append(correct)
          
    print("Epoch: {}/{}...".format(e+1, epochs),
          "Step: {}...".format(counter),
          "Loss: {:.6f}...".format(loss.item()),
          "Val Loss: {:.6f}".format(np.mean(val_losses)),
          "Val Accuracy: {:.6f}".format(np.mean(val_acc)))
    

cuda:0


/usr/local/lib/python3.6/dist-packages/torch/nn/modules/loss.py:498: UserWarning: Using a target size (torch.Size([3, 1])) that is different to the input size (torch.Size([3])) is deprecated. Please ensure they have the same size.
  return F.binary_cross_entropy(input, target, weight=self.weight, reduction=self.reduction)


Epoch: 1/3... Step: 554... Loss: 0.345075... Val Loss: 0.934945 Val Accuracy: 0.333333
Epoch: 1/3... Step: 554... Loss: 0.345075... Val Loss: 0.934945 Val Accuracy: 0.333333
Epoch: 1/3... Step: 554... Loss: 0.345075... Val Loss: 0.836780 Val Accuracy: 0.444444
Epoch: 1/3... Step: 554... Loss: 0.345075... Val Loss: 0.861321 Val Accuracy: 0.416667
Epoch: 1/3... Step: 554... Loss: 0.345075... Val Loss: 0.817147 Val Accuracy: 0.466667
Epoch: 1/3... Step: 554... Loss: 0.345075... Val Loss: 0.787698 Val Accuracy: 0.500000
Epoch: 1/3... Step: 554... Loss: 0.345075... Val Loss: 0.808733 Val Accuracy: 0.476190
Epoch: 1/3... Step: 554... Loss: 0.345075... Val Loss: 0.787698 Val Accuracy: 0.500000
Epoch: 1/3... Step: 554... Loss: 0.345075... Val Loss: 0.771337 Val Accuracy: 0.518519
Epoch: 1/3... Step: 554... Loss: 0.345075... Val Loss: 0.787698 Val Accuracy: 0.500000
Epoch: 1/3... Step: 554... Loss: 0.345075... Val Loss: 0.774312 Val Accuracy: 0.515152
Epoch: 1/3... Step: 554... Loss: 0.345075..